In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from datetime import datetime

#
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = pd.read_csv('train.csv')

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#only 10 rows with missing data/ maybe ok to just drop
data[data['MISSING_DATA'] == True].shape[0]

10

In [5]:
def polyline_to_trip_duration(polyline):
    return max(polyline.count("[") - 2, 0) * 15 #subtracting 2 because one is for the opening bracket?

# This code creates a new column, "LEN", in our dataframe. The value is
# the (polyline_length - 1) * 15, where polyline_length = count("[") - 1
data["LEN"] = data["POLYLINE"].apply(polyline_to_trip_duration)

**Making time columns**

In [6]:
def parse_time(x):
    dt = datetime.fromtimestamp(x["TIMESTAMP"])
    return dt.year, dt.month, dt.day, dt.hour, dt.weekday()

In [7]:
data[["YR", "MON", "DAY", "HR", "WK"]] = (
    data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
)

In [8]:
train, valid = train_test_split(data, train_size = 0.7)

In [9]:
del data  #clearing memory since this dataframe is now redundant

In [10]:
%system free -m

['              total        used        free      shared  buff/cache   available',
 'Mem:         385381       37849      104195          92      243336      346327',
 'Swap:             0           0           0']

**Add interaction terms** ?
- day * hour (both categorical)
- hour * origin stand
- day_type * origin stand

## Data engineering
- Should all the features be categorical? Seems wrong

In [22]:
train.columns
#independent variables are: call_type, origin_call, origin_stand, yr, mon, day, hr, week
#all categorical?

# most origin call are NaN (~78% of training data). probably drop, not sure how we'd infer it
# about 52% of origin_stand are NaN. should we impute?
#no NaN in call_type


Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE', 'LEN', 'YR', 'MON',
       'DAY', 'HR', 'WK'],
      dtype='object')

In [30]:
train[train['ORIGIN_STAND'].isna()].shape[0] / train.shape[0]

0.5282633621413164

In [11]:
X_train = train[["YR", "MON", "DAY", "HR", "WK", 'ORIGIN_STAND']] #add back in call_type once we fix that
Y_train = train["LEN"]

In [12]:
##why is this not working?
#X_train['CALL_TYPE'] = X_train['CALL_TYPE'].astype(str) 

In [13]:
# #probably want to do an embedding instead
# encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# # Fit the encoder on the training data
# encoder.fit(X_train[X_train.columns])
# X_train_OH = encoder.transform(X_train[X_train.columns])

# X_train = torch.Tensor(X_train_OH).to(device)

In [17]:
train.columns

Index(['TRIP_ID', 'CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TAXI_ID',
       'TIMESTAMP', 'DAY_TYPE', 'MISSING_DATA', 'POLYLINE', 'LEN', 'YR', 'MON',
       'DAY', 'HR', 'WK'],
      dtype='object')

In [64]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, dropped_columns):
        self.data = dataframe.drop(columns=dropped_columns)#.values
        self.targets = dataframe["LEN"].astype(float).values
        print(self.data.columns)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        print(self.data[idx])
        x = torch.tensor(self.data[idx], dtype=torch.float32)
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
        return x, y


dataset = CustomDataset(train, dropped_columns=['LEN', 'POLYLINE', 
                                                'MISSING_DATA', 'TRIP_ID', 'ORIGIN_CALL',
                                               "DAY_TYPE", 'TAXI_ID'])

Index(['CALL_TYPE', 'ORIGIN_STAND', 'TIMESTAMP', 'YR', 'MON', 'DAY', 'HR',
       'WK'],
      dtype='object')


In [92]:
# class PandasDataset(torch.utils.data.Dataset):
#     def __init__(self, dataframe):
#         dataframe = dataframe[['CALL_TYPE', 'ORIGIN_STAND', 'YR', 'MON', 'DAY', 'HR',
#        'WK']]
#         dataframe['CALL_TYPE'] = dataframe['CALL_TYPE']
#         self.data = dataframe[['CALL_TYPE', 'ORIGIN_STAND', 'YR', 'MON', 'DAY', 'HR',
#        'WK']]
        
#     def __len__(self):
#         return len(self.data)
    
#     def __getitem__(self, idx):
#         x = self.data.iloc[idx, :]
#         print((x.values))
#         return torch.from_numpy(x.values)


In [93]:
class PandasDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe):
        self.targets = dataframe["LEN"].astype(float).values
        self.data = dataframe[['CALL_TYPE', 'ORIGIN_STAND', 'YR', 'MON', 'DAY', 'HR', 'WK']]
        self.call_type_mapping = {'A': 0, 'B': 1, 'C': 2}
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data.iloc[idx, :].copy()
        x['CALL_TYPE'] = self.call_type_mapping[x['CALL_TYPE']]
        
        y = torch.tensor(self.targets[idx], dtype=torch.float32)
     
        return torch.from_numpy(x.values.astype(np.float32)), y

dataset = PandasDataset(train)

In [88]:
#dataset = torch.utils.data.tensorDataset(X_train, torch.from_numpy(Y_train.values))

In [94]:
batch_size = 32
train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)

# First model

In [102]:
class BasicModel(nn.Module):
    def __init__(self, num_call_types, num_origin_stands, embedding_dim, hidden_size):
        super(BasicModel, self).__init__()
        
        # Embedding layers
        self.call_type_embedding = nn.Embedding(num_call_types, embedding_dim)
        self.origin_stand_embedding = nn.Embedding(num_origin_stands, embedding_dim)
        
        # Other layers
        self.encoder = nn.Sequential(
            nn.Linear(embedding_dim + 4, hidden_size),  # 4 is for [hour, day, month, year]
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
    
    def forward(self, hour, day, month, year, call_type, origin_stand):
        # Embedding lookup
        call_type_embedded = self.call_type_embedding(call_type)
        origin_stand_embedded = self.origin_stand_embedding(origin_stand)
        
        hour = hour.unsqueeze(1)
        day = day.unsqueeze(1)
        month = month.unsqueeze(1)
        year = year.unsqueeze(1)
        
        # Concatenate features
        features = torch.cat([hour, day, month, year, call_type_embedded, origin_stand_embedded], dim=1)
        
        # Pass through the encoder
        output = self.encoder(features)
        
        return output


In [103]:
#not sure about input size
num_call_types = train['CALL_TYPE'].nunique() 
num_origin_stands = train['ORIGIN_STAND'].nunique() 
embedding_dim = 10 #arbitrary
hidden_size = 64 #arbitrary

model = BasicModel(num_call_types, 
                   num_origin_stands, embedding_dim, 
                   hidden_size).to(device)

loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)

In [104]:
#deep network (not lstm)

for epoch in range(5):
    
    running_loss = 0
    n = 0
    
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for batch_idx, (x_batch, y_batch) in enumerate(train_loader):
        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        
        
        ['CALL_TYPE', 'ORIGIN_STAND', 'YR', 'MON', 'DAY', 'HR',
       'WK']
        
        call_type = x_batch[:, 0].long()  # hour is the first column (index 0)
        origin_stand = x_batch[:, 1].long()  # Assuming day is the second column (index 1)
        year = x_batch[:, 2]  # Assuming month is the third column (index 2)
        month = x_batch[:, 3]  # Assuming year is the fourth column (index 3)
        day = x_batch[:, 4]  # Assuming call_type is the fifth column (index 4)
        hour = x_batch[:, 5]  # Assuming origin_stand is the sixth column (index 5)
        week = x_batch[:, 6] #week is 7th col

        y_batch_pred = model(hour, day, month, year, call_type, origin_stand)


        loss = loss_fn(y_batch_pred[0], y_batch.float())
        running_loss += loss.item()
        n += 1

        # backwards steps
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
    print(f"Epoch {epoch + 1} loss: {running_loss/n}")

Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x24 and 14x64)

In [20]:
hidden_state = (torch.zeros(1, hidden_size).to(device),
                torch.zeros(1, hidden_size).to(device))

# https://stackoverflow.com/questions/51735001/how-to-include-batch-size-in-pytorch-basic-example
for epoch in range(5):
    
    running_loss = 0
    n = 0
    
    print(f"Epoch {epoch + 1}\n-------------------------------")

    # Loop over batches in an epoch using DataLoader
    for id_batch, (x_batch, y_batch) in enumerate(train_loader):
        
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

        y_batch_pred = model(x_batch, hidden_state)

        loss = loss_fn(y_batch_pred[0], y_batch.float())
        running_loss += loss.item()
        n += 1

        # backwards steps
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        
    print(f"Epoch {epoch + 1} loss: {running_loss/n}")

Epoch 1
-------------------------------
Epoch 1 loss: 474716.41855863854
Epoch 2
-------------------------------
Epoch 2 loss: 474706.96882588905
Epoch 3
-------------------------------
Epoch 3 loss: 474695.4468317422
Epoch 4
-------------------------------
Epoch 4 loss: 474669.8388911181
Epoch 5
-------------------------------
Epoch 5 loss: 474651.3308392597


In [28]:
X_valid = valid[["YR", "MON", "DAY", "HR", "WK", 'ORIGIN_STAND']]

X_valid_OH = encoder.transform(X_valid[X_valid.columns])

X_valid = torch.Tensor(X_valid_OH).to(device)

In [36]:
hidden_state = (torch.zeros(1, hidden_size).to(device),
                torch.zeros(1, hidden_size).to(device))
valid_preds = model(X_valid)#[0]

In [37]:
valid_preds

tensor([[711.6207],
        [708.5093],
        [714.1364],
        ...,
        [714.6372],
        [719.8740],
        [717.0795]], device='cuda:0', grad_fn=<LeakyReluBackward0>)

## Prediction file

In [38]:
test_data = pd.read_csv('test_public.csv')

In [39]:
test_data[["YR", "MON", "DAY", "HR", "WK"]] = (
    test_data[["TIMESTAMP"]].apply(parse_time, axis=1, result_type="expand")
)

In [40]:
X_test = test_data[["YR", "MON", "DAY", "HR", "WK", 'ORIGIN_STAND']]

In [41]:
X_test_OH = encoder.transform(X_test[X_test.columns])

X_test = torch.Tensor(X_test_OH).to(device)

In [44]:
hidden_state = (torch.zeros(1, hidden_size).to(device),
                torch.zeros(1, hidden_size).to(device))
preds = model(X_test).squeeze()

In [46]:
final_preds = pd.DataFrame({'TRIP_ID': test_data['TRIP_ID'], 'TRAVEL_TIME': preds.cpu().detach()})

final_preds.to_csv('test_preds.csv', index = False)

# Old code, can ignore
----------------------------

In [21]:
train_fraction[train_fraction['ORIGIN_STAND'].isna()].shape[0] 

263863

In [11]:
train_fraction = train.sample(500_000)

In [12]:
X = train_fraction[["YR", "MON", "DAY", "HR", "WK", 'ORIGIN_STAND']]
y = train_fraction['LEN']

In [13]:
full_Y = torch.tensor(y.values)

In [14]:
full_X = torch.from_numpy(X.values.astype('float32'))

In [15]:
model_1 = torch.nn.Sequential(
torch.nn.Linear(6, 1))

loss_fn = torch.nn.MSELoss()

# Define optimizer (this will perform your parameter updates use)
lr = 5e-3
opt = torch.optim.SGD(model_1.parameters(), lr=lr)

In [16]:
test = train.sample(500_000)
X_test_pd = test[["YR", "MON", "DAY", "HR", "WK", 'ORIGIN_STAND']]
y_test_pd = test['LEN']

y_test = torch.tensor(y_test_pd.values)
X_test = torch.tensor(X_test_pd.values)



In [ ]:
train_err = []
test_err = []
parameters = []
for i in range(100):
    model_1.train()
  
    y_pred = model_1(full_X) # Compute model outputs
    loss = loss_fn(y_pred, full_Y) # Compute MSE
    opt.zero_grad() # Must reset the gradients every step. Otherwise, gradients from previous iterations would cause interference!!!
    loss.backward() # Compute gradients of all parameters (our model) with respect to our computed loss value (a singular value)
    opt.step() # One gradient step
  
    train_err.append(loss.item())
  
    model_1.eval()
    with torch.no_grad():
        test_err.append(loss_fn(model_1(X_test), y_test).item())

/Users/orenciolli/anaconda3/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([500000])) that is different to the input size (torch.Size([500000, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [17]:
model_1(X_test)

RuntimeError: mat1 and mat2 must have the same dtype

---------------------------------

In [28]:
#train_fraction['POLYLINE'] = train_fraction['POLYLINE'].apply(lambda x: np.array(eval(x)).astype('float64'))

In [16]:
#train_fraction.groupby('ORIGIN_STAND').mean(numeric_only = True).plot(kind = 'bar', y = 'TRIP_LENGTH')

In [19]:
train_fraction['ORIGIN_STAND'].nunique()

63

Predicted important variables: polyline, interaction between timestamp and origin stand

In [9]:
X = np.array(
    train_fraction[['CALL_TYPE', 'ORIGIN_CALL', 'ORIGIN_STAND', 'TIMESTAMP', 'DAY_TYPE', 'POLYLINE']]
)

In [8]:
from ast import literal_eval

In [6]:
first_x = train_fraction['POLYLINE'].iloc[1]

In [35]:
train_fraction['POLYLINE'].iloc[0].dtype

dtype('float64')

In [49]:
test_type = np.array([train_fraction['POLYLINE'].iloc[0], train_fraction['POLYLINE'].iloc[1]])

/var/folders/dl/8n__h2151hd3n3gtfckwb75w0000gn/T/ipykernel_28498/3306076192.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test_type = np.array([train_fraction['POLYLINE'].iloc[0], train_fraction['POLYLINE'].iloc[1]])


In [54]:
a = np.array([
    np.array([0.5, 1.0, 2.0], dtype=np.float16),
    np.array([4.0, 6.0, 8.0], dtype=np.float16),
])

b = torch.from_numpy(a)

In [64]:
np.array(train_fraction['POLYLINE'])[0][0][0]

-8.59509

In [69]:
three_d = np.array([
    np.array([
        np.array([1, 2]),
        np.array([2, 3])
    ]),
    np.array([
        np.array([1, 2]),
        np.array([2, 3])
    ])
])

In [85]:
three_d.shape#[0][0]

(2, 2, 2)

In [91]:
train_fraction['POLYLINE'].iloc[0][0]

array([-8.59509 , 41.146434])

In [71]:
torch.from_numpy(three_d)

tensor([[[1, 2],
         [2, 3]],

        [[1, 2],
         [2, 3]]])

In [89]:
torch.from_numpy(train_fraction['POLYLINE'].to_numpy())

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [17]:
torch.tensor(train_fraction['POLYLINE'].values)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.